In [1]:
%%capture
!pip install transformers datasets soundfile speechbrain accelerate

In [2]:
from huggingface_hub import notebook_login

notebook_login()

# Dataset


In [4]:
import datasets
from datasets import load_dataset

dataset_1 = load_dataset('Cets/audio-logs-vn-1')
dataset_2 = load_dataset('toandev/ThanhPahm-TTS')
dataset_3 = load_dataset('doof-ferb/vais1000')
dataset_4 = load_dataset('doof-ferb/infore1_25hours')
print(dataset_1['train'].features)
print(dataset_2)
print(dataset_3)
print(dataset_4)

train-00000-of-00002.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/261M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4000 [00:00<?, ? examples/s]

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/169M [00:00<?, ?B/s]

...

train-00012-of-00015.parquet:   0%|          | 0.00/551M [00:00<?, ?B/s]

train-00013-of-00015.parquet:   0%|          | 0.00/555M [00:00<?, ?B/s]

train-00014-of-00015.parquet:   0%|          | 0.00/537M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14935 [00:00<?, ? examples/s]

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'text': Value(dtype='string', id=None)}
DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 4000
    })
})
DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 1000
    })
})
DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 14935
    })
})


In [5]:
dataset_2 = dataset_2.rename_column('transcription', 'text')
dataset_2 = dataset_2.cast_column('audio', datasets.Audio(sampling_rate=16000))
dataset_3 = dataset_3.rename_column('transcription', 'text')
dataset_3 = dataset_3.cast_column('audio', datasets.Audio(sampling_rate=16000))
dataset_4 = dataset_4.rename_column('transcription', 'text')
dataset_4 = dataset_4.cast_column('audio', datasets.Audio(sampling_rate=16000))
print(dataset_2['train'].features)
print(dataset_3['train'].features)
print(dataset_4['train'].features)

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'text': Value(dtype='string', id=None)}
{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'text': Value(dtype='string', id=None)}
{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'text': Value(dtype='string', id=None)}


In [ ]:
dataset_1 = dataset_1.map(lambda x: {'text': x['text'].lower(),'speaker_id':1})
dataset_2 = dataset_2.map(lambda x: {'text': x['text'].lower(),'speaker_id':2})
dataset_3 = dataset_3.map(lambda x: {'text': x['text'].lower(),'speaker_id':3})
dataset_4 = dataset_4.map(lambda x: {'text': x['text'].lower(),'speaker_id':4})


dataset_2 = dataset_2['train'].train_test_split(test_size=0.2, shuffle=True)
dataset_3 = dataset_3['train'].train_test_split(test_size=0.2, shuffle=True)
dataset_4 = dataset_4['train'].train_test_split(test_size=0.2, shuffle=True)

print(dataset_1)
print(dataset_2)
print(dataset_3)
print(dataset_4)

Map:   0%|          | 0/1913 [00:00<?, ? examples/s]

Map:   0%|          | 0/479 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/14935 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets, DatasetDict

merged_train = concatenate_datasets([dataset_1['train'], dataset_2['train'], dataset_3['train'], dataset_4['train']])
merged_test = concatenate_datasets([dataset_1['test'], dataset_2['test'], dataset_3['test'], dataset_4['test']])

dataset = DatasetDict({'train': merged_train, 'test':merged_test})
print(dataset)

# Special characters

In [ ]:
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)
tokenizer = processor.tokenizer

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names['train'],
)

dataset_vocab = set(vocabs['train']["vocab"][0] + vocabs['test']['vocab'][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}
new_tokens = list(dataset_vocab - tokenizer_vocab)
print(new_tokens)

In [ ]:
model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))

# Speaker embedding

*! We use a X-vector model for english now, it is possible to train a new X-vector model*

In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embeddings(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

In [ ]:
import numpy as np

def calculate_speaker_embeddings(dataset):
    embeddings = []
    for split in ['train', 'test']:
        for sample in dataset[split]:
            audio = sample['audio']['array']
            embedding = create_speaker_embeddings(audio)
            embeddings.append(embedding)
    speaker_embeddings = np.mean(np.array(embeddings), axis=0)
    print(speaker_embeddings.shape)
    return speaker_embeddings

speaker_embeddings_1 = calculate_speaker_embeddings(dataset_1)
speaker_embeddings_2 = calculate_speaker_embeddings(dataset_2)
speaker_embeddings_3 = calculate_speaker_embeddings(dataset_3)
speaker_embeddings_4 = calculate_speaker_embeddings(dataset_4)

In [ ]:

def prepare_dataset(example, speaker_embeddings_1=speaker_embeddings_1, speaker_embeddings_2=speaker_embeddings_2):
    audio = example["audio"]
    speaker_id = example['speaker_id']
    
    example = processor(
        text=example["text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector (here we used a speaker_embedding for all sample because we have 1 speaker)
    if speaker_id == 1:
        example["speaker_embeddings"] = speaker_embeddings_1
    elif speaker_id == 2:
        example["speaker_embeddings"] = speaker_embeddings_2
    elif speaker_id == 3:
        example["speaker_embeddings"] = speaker_embeddings_3
    elif speaker_id == 4:
        example["speaker_embeddings"] = speaker_embeddings_4
        
    return example


In [ ]:
processed_example = prepare_dataset(dataset['train'][0])
print(list(processed_example.keys()))
print(processed_example["speaker_embeddings"].shape)
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(processed_example["labels"].T)
plt.show()

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names['train'])

# Data collator

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

data_collator = TTSDataCollatorWithPadding(processor=processor)

# Train

In [ ]:
from functools import partial
# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speechT5-vn-1",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-3,
    warmup_steps=50,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=1000,
    eval_steps=100,
    logging_steps=100,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer, EarlyStoppingCallback

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    processing_class=processor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

In [ ]:
trainer.train()